### Data Prep

1. Download list of all the packages. Saved to pypi_package_list_06_03.txt
2. Randomly sample 50k with seed of 42
3. Get daily download data on the 50k from bigquery USING 01A_pypi_bg.sql. Save to bg-results...
4. Recode installer_name to bot or not
5. Filter to packages with 5 or more human downloads
6. Add a column for treatment/control with seed of 42 and save to fin_dataset...

In [ ]:
import pandas as pd
from lxml import html
import requests
import numpy as np
from numpy import random
import csv
import pandas

In [13]:
response = requests.get("https://pypi.org/simple/")

tree = html.fromstring(response.content)

package_list = [package for package in tree.xpath('//a/text()')]

In [14]:
len(package_list)

458274

In [47]:
# Open the file in write mode
with open('pypi_package_list_06_03.txt', 'w') as file:
    # Iterate over the elements of the list
    for item in package_list:
        # Write each element to the file
        file.write(str(item) + '\n')

In [15]:
# Set the random seed
random.seed(42)

# Randomly sample from the list
random_sample = random.choice(package_list, size=50000, replace=False)

In [16]:
len(random_sample)

50000

In [17]:
random_sample

array(['myssml', 'tornform', 'flex-dispatch', ..., 'okonomiyaki', 'chass',
       'pyincore'], dtype='<U122')

In [22]:
# Define the list

# Define the file path
file_path = 'output.csv'

# Open the file in write mode
with open(file_path, 'w', newline='') as csv_file:
    # Create a CSV writer object
    writer = csv.writer(csv_file, quoting=csv.QUOTE_ALL)
    
    # Write the list as a single row
    writer.writerow(random_sample)

In [42]:
### Filter to at least 5 human downloads
bq = pd.read_csv("bq-results-20230603-222533-1685831182985.csv")

In [43]:
bq['bot_or_not'] = bq['installer_name'].replace({'pip': 'human', 
                                                 'Browser': 'bot', 
                                                 'bandersnatch': 'bot',
                                                 'setuptools': 'human',
                                                 'Nexus': 'human',
                                                 'requests': 'bot',
                                                 'devpi': 'bot',
                                                 'pdm': 'human',
                                                 'Homebrew': 'human',
                                                 'Artifactory': 'human',
                                                 'OS': 'human',
                                                 'Bazel': 'human',
                                                 'pex': 'human',
                                                 'conda': 'human',
                                                 'chaquopy': 'human',
                                                })
bq.drop(['installer_name'], axis = 1)
bq['bot_or_not'] = bq['bot_or_not'].fillna('bot')
bq['bot_or_not'].unique()

array(['human', 'bot'], dtype=object)

In [44]:
pip_df = bq[bq['bot_or_not'] == 'human']
package_sum_pip_df = pip_df.groupby('file_project')['downloads'].sum().reset_index()
package_sum_pip_df_5_plus = package_sum_pip_df[package_sum_pip_df['downloads'] >= 5] 
package_sum_pip_df_5_plus.shape

(23965, 2)

In [45]:
np.random.seed(42)

# Create a new column 'Random' with random values of 0 or 1
package_sum_pip_df_5_plus.loc[:, 'treatment'] = np.random.choice([0, 1], size=len(package_sum_pip_df_5_plus), p=[0.8, 0.2])
package_sum_pip_df_5_plus.to_csv("fin_dataset_treat_control.csv", index = False)

/tmp/ipykernel_1934401/3457133780.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  package_sum_pip_df_5_plus.loc[:, 'treatment'] = np.random.choice([0, 1], size=len(package_sum_pip_df_5_plus), p=[0.8, 0.2])


In [46]:
package_sum_pip_df_5_plus['treatment'].value_counts()

treatment
0    19142
1     4823
Name: count, dtype: int64